# Assignment 1-3: Softmax

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## CIFAR-10 Data Loading and Preprocessing

In [3]:
from data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = r'C:\Users\Apple\Downloads\cifar-10-python\cifar-10-batches-py'
    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your need to complete `softmax_loss_naive`.


In [4]:
# First implement the naive softmax loss function with nested loops.
# softmax_loss_naive function.

from builtins import range
import numpy as np

def softmax_loss_naive(W, X, y, reg):
    """
    Softmax loss function, naive implementation (with loops)

    Inputs have dimension D, there are C classes, and we operate on minibatches
    of N examples.

    Inputs:
    - W: A numpy array of shape (D, C) containing weights.
    - X: A numpy array of shape (N, D) containing a minibatch of data.
    - y: A numpy array of shape (N,) containing training labels; y[i] = c means
      that X[i] has label c, where 0 <= c < C.
    - reg: (float) regularization strength

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W; an array of same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # TODO: Compute the softmax loss and its gradient using explicit loops.     #
    # Store the loss in loss and the gradient in dW. If you are not careful     #
    # here, it is easy to run into numeric instability. Don't forget the        #
    # regularization!                                                           #
    #############################################################################
    # *****START OF YOUR CODE*****
    N, D=X.shape
    C =W.shape[1]
    #Loop 
    for i in range(N):
        #Scores
        scores=X[i].dot(W)
        # Apply numeric stability trick by subtracting the maximum score
        # from the scores to prevent large exponentials
        shift_scores = scores - np.max(scores)

        # Compute the softmax probabilities
        softmax = np.exp(shift_scores) / np.sum(np.exp(shift_scores))

        # Compute the loss for the current example
        loss += -np.log(softmax[y[i]])

        # Compute the gradient for each class
        for j in range(C):
            if j == y[i]:
                dW[:, j] += (softmax[j] - 1) * X[i]
            else:
                dW[:, j] += softmax[j] * X[i]

    # Average the loss and add regularization
    loss /= N
    loss += 0.5 * reg * np.sum(W * W)

    # Average the gradient and add regularization
    dW /= N
    dW += reg * W


    pass

    # *****END OF YOUR CODE*****

    return loss, dW

In [5]:
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.302374
sanity check: 2.302585


In [6]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: 0.867372 analytic: 0.867372, relative error: 4.928925e-08
numerical: 1.739955 analytic: 1.739955, relative error: 2.385084e-09
numerical: -0.803407 analytic: -0.803407, relative error: 2.506228e-08
numerical: 1.213730 analytic: 1.213730, relative error: 3.486475e-08
numerical: 0.818036 analytic: 0.818036, relative error: 2.983972e-08
numerical: 1.710704 analytic: 1.710704, relative error: 1.442223e-08
numerical: 0.939579 analytic: 0.939579, relative error: 5.660503e-08
numerical: -1.364404 analytic: -1.364404, relative error: 1.976721e-09
numerical: -1.946085 analytic: -1.946085, relative error: 7.385314e-09
numerical: 0.193766 analytic: 0.193766, relative error: 2.761351e-07
numerical: -1.865359 analytic: -1.865359, relative error: 1.505835e-08
numerical: -0.661890 analytic: -0.661890, relative error: 8.096412e-08
numerical: -0.366571 analytic: -0.366571, relative error: 2.059411e-08
numerical: -5.213557 analytic: -5.213557, relative error: 2.258516e-09
numerical: 1.501143 

In [7]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.

def softmax_loss_vectorized(W, X, y, reg):
    """
    Softmax loss function, vectorized version.

    Inputs and outputs are the same as softmax_loss_naive.
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    #############################################################################
    # TODO: Compute the softmax loss and its gradient using no explicit loops.  #
    # Store the loss in loss and the gradient in dW. If you are not careful     #
    # here, it is easy to run into numeric instability. Don't forget the        #
    # regularization!                                                           #
    #############################################################################
    # *****START OF YOUR CODE*****
# Number of training examples
    num_train = X.shape[0]

    # Compute the scores
    scores = X.dot(W)

    # Normalize the scores for numerical stability
    scores -= np.max(scores, axis=1, keepdims=True)

    # Compute the softmax probabilities
    softmax_probs = np.exp(scores) / np.sum(np.exp(scores), axis=1, keepdims=True)

    # Compute the loss
    correct_class_probs = softmax_probs[np.arange(num_train), y]
    loss = -np.sum(np.log(correct_class_probs))
    loss /= num_train
    loss += reg * np.sum(W * W)  # Regularization term

    # Compute the gradient
    softmax_probs[np.arange(num_train), y] -= 1  # Subtract 1 from the probabilities of the correct classes
    dW = X.T.dot(softmax_probs)
    dW /= num_train
    dW += 2 * reg * W  # Regularization gradient
    pass

    # *****END OF YOUR CODE*****

    return loss, dW

In [8]:
# The two versions should compute the same results, but the vectorized version should be
# much faster.

tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))


tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.302374e+00 computed in 0.254773s
vectorized loss: 2.302374e+00 computed in 0.064874s
Loss difference: 0.000000
Gradient difference: 0.000000


### Discussion